In [1]:
# ## 📌 1. Imports
import pandas as pd
from sklearn.linear_model import LinearRegression



In [2]:

# ## 📝 2. Lectura de datasets

# Dataset original sell-in
# sell-in.txt en chunks
chunksize = 10 ** 6
sell_in_chunks = pd.read_csv("C:/Users/juans/Documents/MCD/1-materias/laboratorio III/TP_final/data/sell-in.txt", delimiter='\t', chunksize=chunksize)

# Procesar sell-in por chunks y filtrar columnas necesarias
sell_in_list = []
for chunk in sell_in_chunks:
    df_chunk = chunk
    # df_chunk = chunk[['periodo', 'product_id', 'tn']]
    sell_in_list.append(df_chunk)

df = pd.concat(sell_in_list)
print(df.head())


   periodo  customer_id  product_id  plan_precios_cuidados  cust_request_qty  \
0   201701        10234       20524                      0                 2   
1   201701        10032       20524                      0                 1   
2   201701        10217       20524                      0                 1   
3   201701        10125       20524                      0                 1   
4   201701        10012       20524                      0                11   

   cust_request_tn       tn  
0          0.05300  0.05300  
1          0.13628  0.13628  
2          0.03028  0.03028  
3          0.02271  0.02271  
4          1.54452  1.54452  


In [3]:
# Dataset de productos a predecir (780 registros)
productos_apredecir = pd.read_excel("C:/Users/juans/Documents/MCD/1-materias/laboratorio III/TP_final/data/product_id_apredecir201912.xlsx")
# Verificar estructura
print(productos_apredecir.head())
productos_apredecir.shape


   product_id
0       20001
1       20002
2       20003
3       20004
4       20005


(780, 1)

In [4]:
# ## 📝 3. Agrupar sell-in por product_id y periodo, sumando tn

df_grouped = df.groupby(['product_id', 'periodo'], as_index=False)['tn'].sum()
print(df_grouped.shape)
df_grouped.head()


(31243, 3)


,product_id,periodo,tn
0,20001,201701,934.77222
1,20001,201702,798.01620
2,20001,201703,1303.35771
3,20001,201704,1069.96130
4,20001,201705,1502.20132


In [5]:
cantidad_productos = df_grouped['product_id'].value_counts()
cantidad_productos.shape

(1233,)

In [6]:
cantidad_meses = df_grouped['periodo'].value_counts()
cantidad_meses.shape

(36,)

In [7]:
# ## 📝 4. Crear campo 'clase' (tn del periodo +2)

df_grouped = df_grouped.sort_values(['product_id', 'periodo'])
df_grouped['clase'] = df_grouped.groupby('product_id')['tn'].shift(-2)


In [8]:
def generar_lags_y_media(df_grouped, col_producto='product_id', col_periodo='periodo', col_tn='tn', num_lags=11):
    """
    Calcula 11 lags del campo 'tn' y su promedio, por producto y ordenado por período.

    Args:
        df_grouped (pd.DataFrame): DataFrame original con columnas product_id, periodo y tn.
        col_producto (str): Nombre de la columna que identifica el producto.
        col_periodo (str): Nombre de la columna que indica el período.
        col_tn (str): Nombre de la columna con los valores (toneladas).
        num_lags (int): Número de lags a calcular.

    Returns:
        pd.DataFrame: El mismo DataFrame con columnas tn_1 a tn_11 y tn_media.
    """
    df_grouped = df_grouped.sort_values([col_producto, col_periodo]).copy()

    # Calcular los lags tn_1 a tn_11
    for lag in range(1, num_lags + 1):
        df_grouped[f'{col_tn}_{lag}'] = df_grouped.groupby(col_producto)[col_tn].shift(lag)

    # Calcular la media de los lags (ignora NaN si no hay historia suficiente)
    lag_columns = [f'{col_tn}_{lag}' for lag in range(1, num_lags + 1)]
    df_grouped[f'{col_tn}_media'] = df_grouped[lag_columns].mean(axis=1, skipna=True)

    return df_grouped


In [9]:
df_grouped = generar_lags_y_media(df_grouped, col_producto='product_id', col_periodo='periodo', col_tn='tn', num_lags=11)
df_grouped.head(3)

,product_id,periodo,tn,clase,tn_1,tn_2,tn_3,tn_4,tn_5,tn_6,tn_7,tn_8,tn_9,tn_10,tn_11,tn_media
0,20001,201701,934.77222,1303.35771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20001,201702,798.01620,1069.96130,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,934.77222
2,20001,201703,1303.35771,1502.20132,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,866.39421


In [10]:
# Imputar NaNs en los lags con el promedio por producto
lag_columns = [f"tn_{lag}" for lag in range(1, 12)] + ["tn_media"]

for col in lag_columns:
    df_grouped[col] = df_grouped.groupby("product_id")[col].transform(
        lambda x: x.fillna(x.mean())
    )


In [11]:
# Paso 1: Definir los productos mágicos
magicos = [20002, 20003, 20006, 20010, 20011, 20018, 20019, 20021,
           20026, 20028, 20035, 20039, 20042, 20044, 20045, 20046, 20049,
           20051, 20052, 20053, 20055, 20008, 20001, 20017, 20086, 20180,
           20193, 20320, 20532, 20612, 20637, 20807, 20838]

# Paso 2: Filtrar dataset del período 201812 y productos mágicos
dataset_training = df_grouped[
    (df_grouped['periodo'] == 201812) &
    (df_grouped['product_id'].isin(magicos))
]

# Paso 3: Eliminar filas con valores nulos en las variables necesarias
vars_input = ['tn'] + [f'tn_{i}' for i in range(1, 12)]
dataset_training = dataset_training.dropna(subset=vars_input + ['clase'])

# Paso 4: Definir variables independientes (X) y variable dependiente (y)
X = dataset_training[vars_input]
y = dataset_training['clase']

# Paso 5: Entrenar modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X, y)

# Paso 6: Mostrar coeficientes
coeficientes = pd.DataFrame({
    'variable': ['intercept'] + vars_input,
    'coeficiente': [modelo.intercept_] + list(modelo.coef_)
})

print(coeficientes)


dataset_training.head()

     variable  coeficiente
0   intercept     0.441467
1          tn    -0.001339
2        tn_1     0.236558
3        tn_2     0.178208
4        tn_3    -0.060031
5        tn_4    -0.161875
6        tn_5    -0.007775
7        tn_6     0.151936
8        tn_7     0.043933
9        tn_8     0.142839
10       tn_9     0.103804
11      tn_10     0.119211
12      tn_11     0.073671


,product_id,periodo,tn,clase,tn_1,tn_2,tn_3,tn_4,tn_5,tn_6,tn_7,tn_8,tn_9,tn_10,tn_11,tn_media
23,20001,201812,1486.68669,1259.09363,1813.01511,2295.19832,1438.67455,1800.96168,1470.41009,1150.79169,1293.89788,1251.28462,1856.83534,1043.76470,1169.07532,1507.628118
59,20002,201812,1009.45458,1043.01349,1766.81068,1378.49032,954.23575,1161.88430,977.40239,1033.82845,1103.39191,999.20934,966.86044,712.00087,984.80167,1094.446920
95,20003,201812,769.82869,758.32657,1206.91773,1313.34211,912.34156,955.97079,656.22700,660.73323,784.35885,765.47838,778.55594,788.30749,907.56304,884.526920
203,20006,201812,407.75925,479.99914,566.66809,513.15472,478.04388,615.70617,515.20419,468.15260,865.28861,748.44391,862.19361,588.56272,470.33785,608.341486
275,20008,201812,426.32899,476.98787,433.50170,532.45644,436.96269,554.82147,526.38149,554.57063,707.59267,691.53246,765.98901,506.25385,469.29224,561.759514


In [12]:
# %% [markdown]
# ## 📝 7. Entrenar regresión lineal

# %%
X_cols = ['tn'] + [f'tn_{i}' for i in range(1,12)]
X_train = dataset_training[X_cols]
y_train = dataset_training['clase']

model = LinearRegression()
model.fit(X_train, y_train)



,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [13]:
# %% [markdown]
# ## 📝 8. Preparar dataset de predicción filtrado por productos_apredecir

# %%
# Filtrar solo los product_id a predecir
df_pred = df_grouped[df_grouped['product_id'].isin(productos_apredecir['product_id'])]

# Tomar el último periodo disponible para cada product_id
df_pred = df_pred.sort_values(['product_id', 'periodo']).groupby('product_id').tail(1)


In [14]:
# %% [markdown]
# ## 📝 9. Generar predicciones

# %%
# Filtrar registros completos
df_pred_completos = df_pred.dropna(subset=X_cols).copy()

# Predecir
df_pred_completos['tn'] = model.predict(df_pred_completos[X_cols])


In [15]:
df_pred_completos.shape

(656, 16)

In [16]:
# Suma de toneladas predichas
df_pred_completos = df_pred[df_pred[['tn'] + [f'tn_{i}' for i in range(1, 12)]].notnull().all(axis=1)]
df_pred_completos = df_pred_completos.rename(columns={'tn': 'tn_pred'})
suma_toneladas_completos = df_pred_completos['tn_pred'].sum()
df_pred_completos_final = df_pred_completos[['product_id', 'tn_pred']]
df_pred_completos_final['tn_pred'].sum()

np.float64(23447.85905)

In [17]:
df_pred_incompletos = df_pred[df_pred[['tn'] + [f'tn_{i}' for i in range(1, 12)]].isnull().any(axis=1)]
df_pred_incompletos = df_pred_incompletos[['product_id', 'tn']].rename(columns={'tn': 'tn_pred'})
df_pred_incompletos = df_pred_incompletos[['product_id', 'tn_pred']]
df_pred_incompletos['tn_pred'].sum()


np.float64(1697.39303)

In [18]:
final_predictions = pd.concat([
    df_pred_completos_final,
    df_pred_incompletos
]).rename(columns={'tn_pred': 'tn'}).sort_values('product_id').reset_index(drop=True)


In [19]:
final_predictions['tn'].sum()

np.float64(25145.252080000002)

In [20]:
final_predictions.head()

,product_id,tn
0,20001,1504.68856
1,20002,1087.30855
2,20003,892.50129
3,20004,637.90002
4,20005,593.24443


In [21]:

# ## 📝 12. Exportar a CSV final

final_predictions.to_csv('predicciones_finales.csv', index=False)
print("✅ Archivo 'predicciones_finales.csv' exportado correctamente con columnas [product_id, tn]")


✅ Archivo 'predicciones_finales.csv' exportado correctamente con columnas [product_id, tn]
